![Logo dell'Università XYZ](img/logo.jpg)

# Advanced Financial Modeling Course

## Introduction

This notebook is part of the teaching material of the Advanced Financial Modeling course! In this series of Jupyter notebooks, we will cover various topics related to financial modeling, including fundamental concepts, practical applications, and hands-on exercises.

### Notebooks Overview

1. [Notebook 1: Curve Building](notebooks/notebook1.ipynb)
2. [Notebook 2: AAD-Greeks](notebooks/notebook2.ipynb)
3. [Notebook 3: Swap AAD Sensitivities](notebooks/notebook3.ipynb)
3. [Notebook 4: Simulation of Extended Short-Rate Models](notebooks/notebook3.ipynb)
4. [Notebook 5: Model Calibration](notebooks/notebook4.ipynb)
5. [Notebook 6: SABR stochastic volatility](notebooks/notebook5.ipynb)
5. [Notebook 7: xVA--CCR simulation](notebooks/notebook5.ipynb)


## Notebook 5: Model Calibration 

This notebook delves into the critical process of calibrating a short-rate model to market volatilities. We'll cover fundamental concepts and techniques essential for achieving accurate model calibration, with a practical example focused on the EUR market.

### Topics Covered

- Introduction to model calibration and its importance in financial modeling.
- Overview of short-rate models commonly used in interest rate modeling.
- Understanding market volatilities and their role in model calibration.
- Techniques for parameter estimation and optimization in model calibration.
- Real-world example: Calibrating a short-rate model to market data for the EUR market.




## References
- Brigo, D., & Mercurio, F. (2006). Interest rate models-theory and practice: with smile, inflation and credit (Vol. 2). Berlin: Springer.

## Import Libraries

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import QuantLib as ql
import datetime
from util import tools
# from collections import namedtuple
import xlwings as xw

import plotly.graph_objects as go

## Import Curve and Volatilities 


In [7]:
wb = xw.Book('data/data.xlsx')
curve_sheet = wb.sheets["curve"]
vol_sheet = wb.sheets["vols"]

today = ql.Date(2,3,2021)#ql.Date.from_date(datetime.datetime.now()) 
calendar = ql.TARGET()
ql.Settings.instance().setEvaluationDate(today)
spot_date = calendar.advance(today, ql.Period('2d'))
daycount = ql.Actual365Fixed()

eur6m_data = curve_sheet.range("A1:B34").options(pd.DataFrame).value
eur6m_dates = list(map(lambda x: ql.Date.from_date(x), eur6m_data.index))
eur6m_discount_curve = ql.DiscountCurve(eur6m_dates, eur6m_data['DF'].values,
                                        daycount, calendar)
yieldTermStructure = eur6m_discount_curve
yieldTermStructure.enableExtrapolation()
handleYieldTermStructure = ql.RelinkableYieldTermStructureHandle(yieldTermStructure)
t0_curve = ql.YieldTermStructureHandle(yieldTermStructure)
index = ql.Euribor6M(handleYieldTermStructure)
daycounter = yieldTermStructure.dayCounter()
eur6m_bpvols_data = vol_sheet.range('A1:N20').options(pd.DataFrame).value

In [8]:
eur6m_bpvols_data

,1y,2y,3y,4y,5y,6y,7y,8y,9y,10y,15y,20y,30y
Oswp Vols,,,,,,,,,,,,,
1m,17.5,18.5,26.1,33.6,42.0,44.4,47.2,49.0,50.8,52.1,57.5,61.6,65.4
2m,17.7,18.6,24.8,31.4,38.7,41.3,44.0,46.1,48.1,49.3,54.5,58.0,61.0
3m,17.2,18.2,24.4,31.1,37.7,40.7,43.4,45.5,47.3,48.6,53.3,56.8,59.3
6m,18.9,20.8,26.1,31.8,37.0,39.6,41.9,44.0,45.7,47.3,51.7,54.5,56.5
9m,19.9,22.3,27.2,32.6,37.3,40.0,42.4,44.2,46.0,47.3,51.6,54.2,55.9
1y,21.2,23.6,28.2,33.0,37.7,40.7,42.8,44.7,46.4,47.8,51.6,53.8,55.6
18m,23.4,26.5,31.0,35.2,39.8,42.4,44.3,46.1,47.7,49.0,52.2,54.4,55.9
2y,26.7,30.8,35.1,38.3,42.0,44.0,46.0,47.6,49.0,50.3,52.8,54.7,55.9
3y,35.0,38.4,41.2,43.6,46.0,47.7,49.2,50.3,51.3,52.1,53.6,54.9,55.5


## GSR Time-Dependent  Volatility -- Calibration

In [9]:
model = tools.calibrateGSR(eur6m_bpvols_data, today, t0_curve, index, calendar)

----------------------------------------------------------------------------------
    Model Price    Market Price     Implied Vol      Market Vol       Rel Error
----------------------------------------------------------------------------------
        0.02199         0.02373         0.00606         0.00654        -0.07342
        0.02031         0.01912         0.00616         0.00580         0.06230
        0.01626         0.01627         0.00532         0.00533        -0.00049
        0.02230         0.02230         0.00517         0.00517         0.00002
        0.01681         0.01681         0.00473         0.00473        -0.00006
        0.01956         0.01956         0.00478         0.00478         0.00001
        0.02157         0.02157         0.00477         0.00477         0.00005
        0.02207         0.02207         0.00476         0.00476        -0.00008
        0.02447         0.02447         0.00492         0.00492        -0.00022
        0.02440         0.02439   

print to excel:

In [13]:
helpersmatr = tools.helpersMatr(model, eur6m_bpvols_data, t0_curve, index)
modelvalue = np.zeros((len(helpersmatr), len(helpersmatr[0])))
implied_vol = np.zeros((len(helpersmatr), len(helpersmatr[0])))
relative_error = np.zeros((len(helpersmatr), len(helpersmatr[0])))
try:
    for i in range(len(helpersmatr)):
        for j in range(len(helpersmatr[i])):
            swaption = helpersmatr[i][j]
            modelvalue[i,j] = swaption.modelValue()
            market_vol = helpersmatr[i][j].volatility().value()
            implied_vol[i,j] = swaption.impliedVolatility(modelvalue[i,j],
                                              1e-5, 50, 0.0, 0.50)
            rel_error2 = (implied_vol[i,j]/market_vol-1.0)
            relative_error[i,j] = rel_error2
except:
    print("Exception ")

df_implied = pd.DataFrame(implied_vol)
sheet_results = wb.sheets["model_surface"]
sheet_results.range("A1").value = df_implied
sheet_results.range("R25").value = abs(relative_error)

In [11]:

vol_maturities = [1,2,3,4,5,6,7,8,9,10,15,20,30]
vol_tenors = [1/12, 2/12, 3/12, 6/12,9/12, 1, 18/12, 2,3,4,5,6,7,10,12,15,20,25,30]
vols = df_implied.values*10000

fig = go.Figure(data=[go.Surface(z=eur6m_bpvols_data)])

fig.update_layout(title='Market Implied Volatility', autosize=True,
                  width=800, height=700)

In [12]:
results_sheet = wb.sheets["model_surface"]

diff_market_model = results_sheet.range('AI24:AV43').options(pd.DataFrame).value
diff_market_model_abs = results_sheet.range('AI2:AV21').options(pd.DataFrame).value

# Create and show figure
fig = go.Figure()

fig.add_trace(go.Heatmap(
                        z=diff_market_model_abs,
                        colorbar=dict(orientation='v')))

fig.show()